# Test 3 - Drug Target Genes

### All CUIs and drugs seen in the Drugs_labeled_for_AEs.txt file (synonym CUIs not included) and present in both Version 1 and Version 2 of PathFX

In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import seaborn as sns
from multiprocessing import Pool
import numpy as np
import pandas as pd
import os
import sys
import warnings
import xlrd
import itertools
import pickle
from functools import partial
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib as mpl
mpl.use('TkAgg')
from matplotlib import backend_bases
import matplotlib.pyplot as plt
import itertools
import re
import ast
from sklearn.linear_model import LinearRegression
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dataframe_image as dfi
import textwrap
import dataframe_image as dfi
sns.set_theme()
np.random.seed(0)

In [2]:
"""
Helper Functions
"""

# function checks if directory exists, if not it constructs it
def check_directory_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
def read_csv_file(file_path, input_sep=',', input_delimiter=None, input_index_col=None, input_dtype=None, input_delim_whitespace=False, input_low_memory=True):
    return pd.read_csv(filepath_or_buffer=file_path, sep=input_sep, delimiter=None, index_col=input_index_col, dtype=input_dtype, delim_whitespace=input_delim_whitespace, low_memory=input_low_memory)

# funcion saves DataFrame or list to as a textfile
def save_to_csv_file(output_folder_dest, df, file_name, input_index=False, export_style=True, show_print_out=False):
    output_file_name = output_folder_dest + file_name
    output_csv_filename = output_file_name + ".csv"
    output_png_filename = output_file_name + ".png"
    df.to_csv(output_csv_filename, index=input_index)
    style_df = df.copy(deep=True)
    if (export_style):
        #cm = sns.light_palette("seagreen", as_cmap=True)
        #style_df = df.copy(deep=true).style.background_gradient(cmap=cm)
        style_df = df.style.background_gradient()
    try:
        dfi.export(style_df, output_png_filename)
    except ValueError:
        truncated_df = df.copy(deep=True).iloc[0:30][df.columns[0:15]]
        #truncated_style_df = truncated_df.style.background_gradient()
        dfi.export(truncated_df, output_png_filename)
        
    if(show_print_out):
        print("Constructed and saved", output_filename)

# Read in Pickle File
def read_pickle_file(file_path):
    if not os.path.exists(file_path):
        sys.exit("Can't locate input file %s" % file_path)
    return pd.read_pickle(file_path)

In [3]:
parent_folder = "output_modify_all_pathfx_version_drug_association_super_table/"
test = "all_version_intersecting_and_fda_data"
test_folder = parent_folder + test + "/"

# all associated records
test_all_drugs_super_association_df = read_csv_file(test_folder + test + "_all_drugs_super_association_table.csv")
test_all_drugs_super_association_df

,cui,drug,drugbank_ID,phenotype_v1,association_exists_v1,probability_v1,assoc_in_neigh_v1,assoc_in_intom_v1,benjamini_hochberg_v1,neigh_genes_v1,...,drug_targets_v1_size,drug_targets_v2_size,version_difference_drug_target_gene_lst,version_difference_unique_drug_target_gene_lst,version_difference_drug_target_gene_lst_percentage,version_difference_unqiue_drug_target_gene_lst_percentage,version_difference_intersecting_drug_target_gene_lst_percentage,ratio_drug_target_gene_over_neigh_gene_lst_v1_size,ratio_drug_target_gene_over_neigh_gene_lst_v2_size,version_difference_ratio_drug_target_gene_over_neigh_gene_lst_size
0,C0243026,abatacept,DB01281,Sepsis,Yes,1.079882e-06,5,602,0.000060,"['CD80', 'CD86', 'FYN', 'PIK3CA', 'PIK3R1']",...,2,2,0,0,0.000314,0.000000,0.000314,0.400000,0.666667,0.266667
1,C0027051,acamprosate,DB00659,Myocardial Infarction,Yes,2.347811e-09,21,1019,0.000059,"['ADRA1D', 'AGT', 'AVPR1A', 'BDKRB1', 'BDKRB2'...",...,1,2,1,1,0.000144,-0.000419,-0.000419,0.047619,0.117647,0.070028
2,C0038358,acamprosate,DB00659,Gastric ulcer,Yes,4.382202e-09,8,116,0.000063,"['BDKRB1', 'BDKRB2', 'CCK', 'EDN1', 'GAST', 'G...",...,1,2,1,1,0.012656,0.002018,0.002018,0.125000,0.333333,0.208333
3,C0027051,adenosine,DB00640,Myocardial Infarction,Yes,1.735121e-10,36,1019,0.000064,"['ADM', 'ADORA1', 'ADRA2B', 'ADRB1', 'ADRB3', ...",...,6,6,0,0,-0.002514,0.000000,-0.002514,0.166667,0.171429,0.004762
4,C0030305,aldesleukin,DB00041,Pancreatitis,Yes,1.519662e-04,5,277,0.000320,"['CYP2E1', 'IL2RA', 'PTGS2', 'TP53', 'XDH']",...,8,8,0,0,-0.000909,0.000000,-0.000909,1.600000,2.000000,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,C0020473,ziprasidone,DB00246,"Hyperlipidemia, combined, 1",Yes,6.202761e-11,33,293,0.000069,"['ADM', 'ADRB1', 'ADRB2', 'ADRB3', 'AGT', 'AGT...",...,29,27,-2,-2,0.017907,0.000916,0.023817,0.878788,1.687500,0.808712
351,C0032285,ziprasidone,DB00246,Pneumonia,Yes,5.466938e-11,44,471,0.000064,"['ADORA2B', 'ADORA3', 'ADRB2', 'C3', 'C3AR1', ...",...,29,27,-2,-2,-0.007354,-0.000115,-0.002993,0.659091,0.870968,0.211877
352,C0686347,ziprasidone,DB00246,Tardive Dyskinesia,Yes,4.269929e-11,18,64,0.000059,"['APOE', 'ARRB2', 'CNR1', 'CYP1A2', 'CYP2D6', ...",...,29,27,-2,-2,0.086875,0.004375,0.113750,1.611111,3.375000,1.763889
353,C0027051,zolmitriptan,DB00315,Myocardial Infarction,Yes,8.673792e-10,29,1019,0.000055,"['ADORA1', 'ADORA3', 'ADRA2B', 'AGT', 'AGTR2',...",...,6,6,0,0,-0.002514,0.000000,-0.002514,0.206897,0.206897,0.000000


In [4]:
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52', '#BCBD22', '#9D755D', '#BAB0AC', '#F7E1A0', '#17BECF', '#1616A7']

adverse_event_lst = ['Sepsis', 'Myocardial Infarction', 'Gastric ulcer', 'Pancreatitis', 'Pneumonia', 'Cerebral Infarction', 'Hemorrhage', 'Deep Vein Thrombosis', 'Edema', 'Tardive Dyskinesia', 'Delirium', 'Hyperlipidemia', 'Thrombocytopenia', 
                     'Peripheral Neuropathy', 'Pulmonary Edema', 'Proteinuria']

color_dict = {adverse_event_lst[i]: colors[i] for i in range(len(colors))}
color_dict

{'Sepsis': '#636EFA',
 'Myocardial Infarction': '#EF553B',
 'Gastric ulcer': '#00CC96',
 'Pancreatitis': '#AB63FA',
 'Pneumonia': '#FFA15A',
 'Cerebral Infarction': '#19D3F3',
 'Hemorrhage': '#FF6692',
 'Deep Vein Thrombosis': '#B6E880',
 'Edema': '#FF97FF',
 'Tardive Dyskinesia': '#FECB52',
 'Delirium': '#BCBD22',
 'Hyperlipidemia': '#9D755D',
 'Thrombocytopenia': '#BAB0AC',
 'Peripheral Neuropathy': '#F7E1A0',
 'Pulmonary Edema': '#17BECF',
 'Proteinuria': '#1616A7'}

In [5]:
drug_target_gene_content_pathfx_version_comparison_info_folder = "drug_target_gene_content_pathfx_version_comparison_info_folder/"
check_directory_exists(drug_target_gene_content_pathfx_version_comparison_info_folder)

In [6]:
import plotly.io as pio
pio.renderers.default = "browser"

In [7]:
df = test_all_drugs_super_association_df.copy(deep=True)

df = df.rename(columns={"phenotype_v2":"Adverse Event", "drug_targets_v1_size": "Number of Drug Target Genes PathFXv1", "drug_targets_v2_size": "Number of Drug Target Genes PathFXv2"})

df_all_records_stats = df[['Number of Drug Target Genes PathFXv1', 'Number of Drug Target Genes PathFXv2']].agg(['mean','std','min','max','sum'])

save_to_csv_file(drug_target_gene_content_pathfx_version_comparison_info_folder, df_all_records_stats, "all_records_drug_target_gene_stats_table", input_index=True)

display(df_all_records_stats)

num_genes_stats_df = df[['Adverse Event', 'Number of Drug Target Genes PathFXv1', 'Number of Drug Target Genes PathFXv2']].groupby("Adverse Event").agg(['mean','std','min','max','sum'])

save_to_csv_file(drug_target_gene_content_pathfx_version_comparison_info_folder, num_genes_stats_df, "num_drug_target_genes_stats_table", input_index=True)

display(num_genes_stats_df)

,Number of Drug Target Genes PathFXv1,Number of Drug Target Genes PathFXv2
mean,12.760563,13.177465
std,10.081399,10.095283
min,1.000000,1.000000
max,44.000000,42.000000
sum,4530.000000,4678.000000


Number of Drug Target Genes PathFXv1                     \
                                                      mean        std min max   
Adverse Event                                                                   
Cerebral Infarction                              10.600000  10.691118   4  29   
Deep Vein Thrombosis                              3.500000   0.707107   3   4   
Delirium                                         19.181818  10.147100   5  34   
Edema                                            17.722222  12.217661   2  44   
Gastric ulcer                                    12.500000   8.528775   1  39   
Hemorrhage                                        3.833333   2.516611   1  11   
Hyperlipidemia                                   17.157895  11.131852   3  38   
Myocardial Infarction                            11.000000   9.683602   1  44   
Pancreatitis                                     13.666667   9.898031   1  39   
Peripheral Neuropathy                             5.000000        NaN   5   5   
Pneumonia                                        12.900000   9.968283   1  44   
Proteinuria                                       6.500000   4.203173   1  11   
Pulmonary Edema                                  16.333333   3.785939  12  19   
Sepsis                                           10.454545   8.566252   1  39   
Tardive Dyskinesia                               19.958333  11.269347   6  44   
Thrombocytopenia                                  9.913043   8.033527   1  27   

                           Number of Drug Target Genes PathFXv2             \
                       sum                                 mean        std   
Adverse Event                                                                
Cerebral Infarction     53                            10.400000   9.710819   
Deep Vein Thrombosis     7                             4.000000   0.000000   
Delirium               211                            21.272727  11.181153   
Edema                  319                            17.055556  10.913953   
Gastric ulcer          325                            12.076923   8.289382   
Hemorrhage              46                             4.750000   3.840573   
Hyperlipidemia         326                            18.578947  11.157040   
Myocardial Infarction  880                            11.325000   9.558315   
Pancreatitis           328                            14.666667   9.809785   
Peripheral Neuropathy    5                             6.000000        NaN   
Pneumonia              903                            12.785714   9.788921   
Proteinuria             26                             7.500000   5.000000   
Pulmonary Edema         49                            20.333333  10.066446   
Sepsis                 345                            10.484848   8.849863   
Tardive Dyskinesia     479                            20.791667  10.475558   
Thrombocytopenia       228                            11.217391   9.356362   

                                    
                      min max  sum  
Adverse Event                       
Cerebral Infarction     4  27   52  
Deep Vein Thrombosis    4   4    8  
Delirium                7  42  234  
Edema                   2  39  307  
Gastric ulcer           2  38  314  
Hemorrhage              1  14   57  
Hyperlipidemia          3  42  353  
Myocardial Infarction   1  42  906  
Pancreatitis            1  42  352  
Peripheral Neuropathy   6   6    6  
Pneumonia               1  42  895  
Proteinuria             1  13   30  
Pulmonary Edema        11  31   61  
Sepsis                  1  38  346  
Tardive Dyskinesia      7  42  499  
Thrombocytopenia        1  31  258

## Distribution of Drug Target Gene List Size between PathFX Versions

In [8]:
df = test_all_drugs_super_association_df.copy(deep=True)

v1 = df["drug_targets_v1_size"]
v2 = df["drug_targets_v2_size"]
fig = go.Figure()
graph_title = "Distribution of Drug Target Gene List Size between PathFX Versions"
fig.add_trace(go.Histogram(x=v1, name='Version 1', marker_color='#FF5557'))
fig.add_trace(go.Histogram(x=v2, name='Version 2', marker_color='#22D646'))
fig.update_layout(title_text=graph_title)
fig.update_xaxes(title_text="Drug Target Gene List Size")
fig.update_yaxes(title_text="Frequency")
fig.update_layout(legend_title_text='PathFX Version')
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.update_layout(showlegend=True, width=2200, height=1000)
fig.write_image(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

a = df["drug_targets_v1_and_drug_targets_v2_set_symmetric_difference_size"]
b = df["drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1_size"]
c = df["drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2_size"]
fig = go.Figure()
graph_title = "Distribution of Unique Drug Target Gene List Size between PathFX Versions"
fig.add_trace(go.Histogram(x=a, name='Symmetric Difference', marker_color='grey'))
fig.add_trace(go.Histogram(x=b, name='Version 1 Unique', marker_color='#FF5557'))
fig.add_trace(go.Histogram(x=c, name='Version 2 Unique', marker_color='#22D646'))
fig.update_layout(title_text=graph_title)
fig.update_xaxes(title_text="Drug Target Gene List Size")
fig.update_yaxes(title_text="Frequency")
fig.update_layout(legend_title_text='PathFX Version')
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.update_layout(showlegend=True, width=2200, height=1000)
fig.write_image(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Difference in -Log(Benjamini-Hochberg) based on Unique Drug Target Gene List

In [9]:
df = test_all_drugs_super_association_df.copy(deep=True)

# Version 1

df = df.sort_values(by="drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1", ascending=False)
graph_title = "Difference in -Log(Benjamini-Hochberg) vs Unique Version 1 Drug Target Gene List"
fig = px.scatter(df, x="version_difference_-log(benjamini_hochberg)", y="drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1", 
                 title=graph_title, color="phenotype_v2",
                 size="abs_version_difference_-log(benjamini_hochberg)", color_discrete_map=color_dict, 
                 hover_data=['cui', 'phenotype_v2', 'drug', 'drugbank_ID', 'drug_targets_v1_and_drug_targets_v2_intersection_size', 
                             'drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1_size', 'drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2_size',
                             "neigh_genes_v1_and_neigh_genes_v2_intersection_size", "neigh_genes_v1_and_neigh_genes_v2_set_difference_for_neigh_genes_v1_size", 
                             "neigh_genes_v1_and_neigh_genes_v2_set_difference_for_neigh_genes_v2_size", "intom_genes_v1_and_intom_genes_v2_intersection_size", 
                             "intom_genes_v1_and_intom_genes_v2_set_difference_for_intom_genes_v1_size", "intom_genes_v1_and_intom_genes_v2_set_difference_for_intom_genes_v2_size"])
fig.update_layout(showlegend=True, width=3500, height=2200)
fig.update_xaxes(title_text="Difference in -Log(Benjamini-Hochberg) between PathFX Versions")
fig.update_yaxes(title_text="Unique Version 1 Drug Target Gene List")
fig.update_layout(legend_title_text='(Phenotype, Drug)')
fig.update_traces(
    hovertemplate="<br>".join([
      "CUI = %{customdata[0]}",
        "Phenotype: %{customdata[1]}",
        "Drug: %{customdata[2]}",
        "DrugBank ID: %{customdata[3]}",
        "Number of Intersecting Drug Target Genes: %{customdata[4]}",
        "Number of Version 1 Unique Drug Target Genes: %{customdata[5]}",
        "Number of Version 2 Unique Drug Target Genes: %{customdata[6]}",
        "Number of Intersecting Neighborhood Genes: %{customdata[7]}", 
        "Number of Version 1 Unique Neighborhood Genes: %{customdata[8]}", 
        "Number of Version 2 Unique Neighborhood Genes: %{customdata[9]}",
        "Number of Intersecting Interactome Genes: %{customdata[10]}",
        "Number of Version 1 Unique Interactome Genes: %{customdata[11]}", 
        "Number of Version 2 Unique Interactome Genes: %{customdata[12]}",
        "(X-Axis)-Difference in -log(Benjamini-Hochberg: %{x}", 
        "(Y-Axis)-Unique PathFX Version 1 Neighborhood Genes: %{y}",
    ])
)
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.write_image(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

# Version 2

df = df.sort_values(by="drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2", ascending=False)
graph_title = "Difference in -Log(Benjamini-Hochberg) vs Unique Version 2 Drug Target Gene List"
fig = px.scatter(df, x="version_difference_-log(benjamini_hochberg)", y="drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2", 
                 title=graph_title, color="phenotype_v2", 
                 size="abs_version_difference_-log(benjamini_hochberg)", color_discrete_map=color_dict, 
                 hover_data=['cui', 'phenotype_v2', 'drug', 'drugbank_ID', 'drug_targets_v1_and_drug_targets_v2_intersection_size', 
                             'drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1_size', 'drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2_size',
                             "neigh_genes_v1_and_neigh_genes_v2_intersection_size", "neigh_genes_v1_and_neigh_genes_v2_set_difference_for_neigh_genes_v1_size", 
                             "neigh_genes_v1_and_neigh_genes_v2_set_difference_for_neigh_genes_v2_size", "intom_genes_v1_and_intom_genes_v2_intersection_size", 
                             "intom_genes_v1_and_intom_genes_v2_set_difference_for_intom_genes_v1_size", "intom_genes_v1_and_intom_genes_v2_set_difference_for_intom_genes_v2_size"])
fig.update_layout(showlegend=True, width=3500, height=2200)
fig.update_xaxes(title_text="Difference in -Log(Benjamini-Hochberg) between PathFX Versions")
fig.update_yaxes(title_text="Unique Version 1 Drug Target Gene List")
fig.update_layout(legend_title_text='(Phenotype, Drug)')
fig.update_traces(
    hovertemplate="<br>".join([
      "CUI = %{customdata[0]}",
        "Phenotype: %{customdata[1]}",
        "Drug: %{customdata[2]}",
        "DrugBank ID: %{customdata[3]}",
        "Number of Intersecting Drug Target Genes: %{customdata[4]}",
        "Number of Version 1 Unique Drug Target Genes: %{customdata[5]}",
        "Number of Version 2 Unique Drug Target Genes: %{customdata[6]}",
        "Number of Intersecting Neighborhood Genes: %{customdata[7]}", 
        "Number of Version 1 Unique Neighborhood Genes: %{customdata[8]}", 
        "Number of Version 2 Unique Neighborhood Genes: %{customdata[9]}",
        "Number of Intersecting Interactome Genes: %{customdata[10]}",
        "Number of Version 1 Unique Interactome Genes: %{customdata[11]}", 
        "Number of Version 2 Unique Interactome Genes: %{customdata[12]}",
        "(X-Axis)-Difference in -log(Benjamini-Hochberg: %{x}", 
        "(Y-Axis)-Unique PathFX Version 1 Neighborhood Genes: %{y}",
    ])
)
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.write_image(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(drug_target_gene_content_pathfx_version_comparison_info_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Gene Record Frequency

In [10]:
gene_frequency_folder = drug_target_gene_content_pathfx_version_comparison_info_folder + "gene_frequency/"
check_directory_exists(gene_frequency_folder)

In [11]:
from ast import literal_eval
from collections import Counter

def gene_content_analysis(column_interest, save_folder, gene_analysis_name):

    df_list = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
    
    df = test_all_drugs_super_association_df.copy(deep=True)
    all_adverse_events = list(df["phenotype_v2"].unique())
    all_adverse_events.sort()

    df = test_all_drugs_super_association_df.copy(deep=True)
    gene_lists = df[column_interest].apply(literal_eval)
    flattened_gene_lists = [item for sublist in gene_lists for item in sublist]
    all_gene_list_dict = dict(Counter(flattened_gene_lists))
    all_gene_list_df = pd.DataFrame.from_dict(all_gene_list_dict, orient='index')
    all_gene_list_df.columns = ["Count"]
    all_gene_list_df = all_gene_list_df.sort_values(by='Count', ascending=False)
    
    df_list[0] = all_gene_list_df

    df = test_all_drugs_super_association_df.copy(deep=True)
    all_gene_count_per_adverse_event_df = pd.DataFrame()
    for adverse_event in all_adverse_events:
        df = test_all_drugs_super_association_df.copy(deep=True)
        df = df[df["phenotype_v2"] == adverse_event]
        gene_lists = df[column_interest].apply(literal_eval)
        flattened_gene_lists = [item for sublist in gene_lists for item in sublist]
        if flattened_gene_lists == []:
            all_gene_count_per_adverse_event_df[adverse_event] = 0
        else:
            all_gene_list_dict = dict(Counter(flattened_gene_lists))
            all_gene_list_for_adverse_event_df = pd.DataFrame.from_dict(all_gene_list_dict, orient='index')
            all_gene_list_for_adverse_event_df.columns = [adverse_event]
            all_gene_count_per_adverse_event_df = pd.concat([all_gene_count_per_adverse_event_df, all_gene_list_for_adverse_event_df], axis=1)
    all_gene_count_per_adverse_event_df = all_gene_count_per_adverse_event_df.fillna(0)
    all_gene_count_per_adverse_event_df = all_gene_count_per_adverse_event_df.reindex(sorted(all_gene_count_per_adverse_event_df.columns), axis=1)
    all_gene_count_per_adverse_event_df = all_gene_count_per_adverse_event_df.sort_index(ascending=True)
    
    df_list[1] = all_gene_count_per_adverse_event_df

    def merge_gene_with_score(x):
        new_list = []
        for gene in x[0]:
            new_list.append([gene,  x[1]])
        return new_list

    df = test_all_drugs_super_association_df.copy(deep=True)
    df[column_interest] = df[column_interest].apply(literal_eval)
    gene_list_scores = df[[column_interest, "version_difference_-log(benjamini_hochberg)"]].apply(merge_gene_with_score, axis=1)
    flattened_gene_list_scores = [item for sublist in gene_list_scores for item in sublist]
    gene_score_df = pd.DataFrame(flattened_gene_list_scores, columns = ['Gene', '-log(BH) Diff'])
    gene_score_df = gene_score_df.sort_values(by="Gene")
    all_record_gene_score_stats_df = gene_score_df.groupby("Gene").agg(['mean','std','min','max','sum'])
    
    df_list[2] = all_record_gene_score_stats_df

    df = test_all_drugs_super_association_df.copy(deep=True)
    gene_score_per_adverse_event_stats_df = pd.DataFrame()
    for adverse_event in all_adverse_events:
        df = test_all_drugs_super_association_df.copy(deep=True)
        df = df[df["phenotype_v2"] == adverse_event]
        df[column_interest] = df[column_interest].apply(literal_eval)
        gene_list_scores = df[[column_interest, "version_difference_-log(benjamini_hochberg)"]].apply(merge_gene_with_score, axis=1)
        flattened_gene_list_scores = [item for sublist in gene_list_scores for item in sublist]
        gene_score_df = pd.DataFrame(flattened_gene_list_scores, columns = ['Gene', adverse_event + " -log(BH) Diff"])
        gene_score_df = gene_score_df.sort_values(by="Gene")
        temp_all_record_gene_score_stats_df = gene_score_df.groupby("Gene").agg(['mean','std','min','max','sum'])
        gene_score_per_adverse_event_stats_df = pd.concat([gene_score_per_adverse_event_stats_df, temp_all_record_gene_score_stats_df], axis=1)
    gene_score_per_adverse_event_stats_df = gene_score_per_adverse_event_stats_df.fillna(0)
    gene_score_per_adverse_event_stats_df = gene_score_per_adverse_event_stats_df.sort_index(ascending=True)

    df_list[3] = gene_score_per_adverse_event_stats_df
    
    data = {"Gene":[], "Count":[], "Adverse Event":[]}
    gene_count_adverse_event_display_df = pd.DataFrame(data)
    temp_df = all_gene_count_per_adverse_event_df.reset_index()
    for adverse_event in all_adverse_events:
        new_temp_df = pd.concat([pd.DataFrame(all_gene_count_per_adverse_event_df.index), pd.DataFrame(temp_df[adverse_event])], axis=1)
        new_temp_df["Adverse Event"] = adverse_event
        new_temp_df = new_temp_df.rename(columns={0: "Gene", adverse_event: "Count"})
        gene_count_adverse_event_display_df = pd.concat([gene_count_adverse_event_display_df, new_temp_df])
    temp_df = all_gene_list_df.reset_index()
    temp_df = temp_df.rename(columns={"index": "Gene"})
    gene_count_adverse_event_display_df = gene_count_adverse_event_display_df.merge(temp_df, how="inner", left_on="Gene", right_on="Gene")
    gene_count_adverse_event_display_df = gene_count_adverse_event_display_df.rename(columns={"Count_x":  "Adverse Event Count", "Count_y":"All Adverse Event Counts"})
    gene_count_adverse_event_display_df = gene_count_adverse_event_display_df.sort_values(by="All Adverse Event Counts", ascending=False)

    def find_mean_adverse_event_score(x):
        return gene_score_per_adverse_event_stats_df[x[2] + " -log(BH) Diff"]["mean"][x[0]]

    def find_mean_score(x):
        return all_record_gene_score_stats_df[('-log(BH) Diff', 'mean')].loc[x[0]] 

    gene_count_adverse_event_display_df["Adverse event mean -log(BH) diff"] = gene_count_adverse_event_display_df.apply(find_mean_adverse_event_score, axis=1) 
    gene_count_adverse_event_display_df["All Adverse events mean -log(BH) diff"] = gene_count_adverse_event_display_df.apply(find_mean_score, axis=1) 
  
    df_list[4] = gene_count_adverse_event_display_df
    
    # Dataframe for drug and -log(BH) record info
    df = test_all_drugs_super_association_df.copy(deep=True)
    all_genes_lst = list(all_gene_list_df.index)
    individual_gene_record_df = pd.DataFrame()
    for gene in all_genes_lst:
        temp_two_df = df[df[column_interest].apply(lambda x: gene in x)]
        temp_two_df = temp_two_df[["drug", "phenotype_v2", "version_difference_-log(benjamini_hochberg)"]]
        temp_two_df["Gene"] = gene  
        individual_gene_record_df = pd.concat([individual_gene_record_df, temp_two_df])
    individual_gene_record_df = individual_gene_record_df[["Gene", "drug", "phenotype_v2", "version_difference_-log(benjamini_hochberg)"]]
    individual_gene_record_df = individual_gene_record_df.rename(columns={"drug": "Drug", "phenotype_v2":"Adverse Event", "version_difference_-log(benjamini_hochberg)":"Difference in -log(BH)"})
    all_gene_record_general_info_df = gene_count_adverse_event_display_df.merge(individual_gene_record_df, how="inner", left_on=["Gene", "Adverse Event"], right_on=["Gene", "Adverse Event"])
    all_gene_record_general_info_df = all_gene_record_general_info_df[["Gene", "Drug", "Adverse Event", "Difference in -log(BH)", "Adverse Event Count", "Adverse event mean -log(BH) diff",
                                                                              "All Adverse Event Counts", "All Adverse events mean -log(BH) diff"]]
    df_list[5] = all_gene_record_general_info_df
    
    gene_average_score_diff_df = all_gene_record_general_info_df.groupby("Gene")[["Difference in -log(BH)"]].mean().reset_index()
    gene_average_score_diff_df.rename(columns={"Difference in -log(BH)":"Mean Difference in -log(BH)"}, inplace=True)

    df_list[6] = gene_average_score_diff_df

    # Saving all dataframes constructed 
    
    parent_folder = save_folder + gene_analysis_name + "/"
    check_directory_exists(parent_folder)

    save_to_csv_file(parent_folder, all_gene_list_df, gene_analysis_name + "_all_genes_frequncy_table", input_index=True)
    save_to_csv_file(parent_folder, all_gene_count_per_adverse_event_df, gene_analysis_name + "_all_genes_frequncy_per_adverse_event_table", input_index=True)
    save_to_csv_file(parent_folder, all_record_gene_score_stats_df, gene_analysis_name + "_all_gene_-log(BH)_score_stats_table", input_index=True)
    save_to_csv_file(parent_folder, gene_score_per_adverse_event_stats_df, gene_analysis_name + "_all_gene_-log(BH)_score_stats_per_adverse_event_table", input_index=True)
    save_to_csv_file(parent_folder, gene_count_adverse_event_display_df, gene_analysis_name + "_all_gene_count_adverse_event_display_table", input_index=False)
    save_to_csv_file(parent_folder, all_gene_record_general_info_df, gene_analysis_name + "_all_gene_record_general_info_table_df", input_index=False)
    save_to_csv_file(parent_folder, gene_average_score_diff_df, gene_analysis_name + "_gene_average_score_diff_df", input_index=False)
    
    # ------ Graphing ------
    
    graph_title = gene_analysis_name + " Difference in -log(BH) for each individual Gene-Drug Record"
    fig = px.scatter(data_frame=all_gene_record_general_info_df, x="Difference in -log(BH)", y="Gene", color="Adverse Event", color_discrete_map=color_dict, 
                     title=graph_title, hover_data=['Drug', 'Adverse Event']).update_xaxes(categoryorder="total descending")
    fig.update_layout(showlegend=True, width=2500, height=1700)
    fig.update_xaxes(title_text="Difference in -log(BH)")
    fig.update_yaxes(title_text="Gene")
    fig.update_layout(legend_title_text='Adverse Event')
    fig.update_traces(
        hovertemplate="<br>".join([
            "Gene: %{y}",
            "Drug: %{customdata[0]}",
            "Adverse Event: %{customdata[1]}",
            "Difference in -log(BH): %{x}"
        ])
    )
    fig.update_traces(marker=dict(line=dict(width=0.25, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.write_image(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
    fig.write_html(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
    # fig.show()

    graph_title = gene_analysis_name + " Gene Frequency"
    fig = px.bar(data_frame=gene_count_adverse_event_display_df, x="Adverse Event Count", y="Gene", color="Adverse Event", color_discrete_map=color_dict, 
                 title=graph_title, hover_data=['All Adverse Event Counts']).update_yaxes(categoryorder="total ascending")
    fig.update_layout(showlegend=True, width=2300, height=1200)
    fig.update_xaxes(title_text="Frequency Across all Records")
    fig.update_yaxes(title_text="Gene")
    fig.update_layout(legend_title_text='Adverse Event')
    fig.update_traces(
        hovertemplate="<br>".join([
            "Gene: %{x}",
            "Adverse Event Gene Count: %{y}",
            "All Adverse Event Gene Counts: %{customdata[0]}",
        ])
    )
    fig.write_image(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
    fig.write_html(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
    # fig.show()

    graph_title = gene_analysis_name + " Mean -log(BH) Difference of a Gene for a specific Adverse Event"
    fig = px.scatter(gene_count_adverse_event_display_df, x="Adverse event mean -log(BH) diff", y="Gene", 
                     title=graph_title, 
                     color="Adverse Event", size="Adverse Event Count", hover_data=["Adverse Event", "Adverse Event Count"], 
                     color_discrete_map=color_dict).update_yaxes(categoryorder="total descending")
    fig.update_layout(showlegend=True, width=2200, height=1000)
    fig.update_xaxes(title_text="Mean -log(BH) difference")
    fig.update_yaxes(title_text="Gene")
    fig.update_layout(legend_title_text='Adverse Event')
    fig.update_traces(
        hovertemplate="<br>".join([
            "Gene: %{y}",
            "Adverse Event: %{customdata[0]}",
            "Adverse Event Count: %{customdata[1]}",
            "Adverse event mean -log(BH) diff: %{x}"
        ])
    )
    fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.write_image(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
    fig.write_html(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
    # fig.show()

    graph_title = gene_analysis_name + " Mean -log(BH) Difference vs Adverse Event Count of a specific Gene"
    fig = px.scatter(gene_count_adverse_event_display_df, x="Adverse event mean -log(BH) diff", y="Adverse Event Count", 
                     title=graph_title, 
                     color="Adverse Event", size="Adverse Event Count", hover_data=["Adverse Event", "Gene"], 
                     color_discrete_map=color_dict).update_yaxes(categoryorder="total descending")
    fig.update_layout(showlegend=True, width=2200, height=1000)
    fig.update_xaxes(title_text="Mean -log(BH) difference")
    fig.update_yaxes(title_text="Adverse Event Count for a specific Gene")
    fig.update_layout(legend_title_text='Adverse Event')
    fig.update_traces(
        hovertemplate="<br>".join([
            "Gene: %{customdata[1]}",
            "Adverse Event: %{customdata[0]}",
            "Adverse Event Count: %{y}",
            "Adverse event mean -log(BH) diff: %{x}"
        ])
    )
    fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.write_image(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
    fig.write_html(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
    # fig.show()
    
    graph_title = gene_analysis_name + " Mean -log(BH) Difference of a Gene across all records"
    gene_average_score_diff_df = gene_average_score_diff_df.sort_values(by='Mean Difference in -log(BH)', ascending=True)
    fig = px.scatter(gene_average_score_diff_df, x="Mean Difference in -log(BH)", y="Gene", 
                     title=graph_title, 
                     color_discrete_map=color_dict).update_xaxes(categoryorder="total descending")
    fig.update_layout(showlegend=True, width=2200, height=1000)
    fig.update_xaxes(title_text="Mean -log(BH) difference")
    fig.update_yaxes(title_text="Gene")
    fig.update_traces(
        hovertemplate="<br>".join([
            "Gene: %{y}",
            "Mean Difference in -log(BH): %{x}"
        ])
    )
    fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.write_image(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
    fig.write_html(parent_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
    # fig.show()
    
    return df_list

In [ ]:
output_dfs_list = gene_content_analysis("drug_targets_v1", gene_frequency_folder, "drug_target_genes_version_1")
output_dfs_list = gene_content_analysis("drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v1", gene_frequency_folder, "drug_target_genes_version_1_only")
output_dfs_list = gene_content_analysis("drug_targets_v1", gene_frequency_folder, "drug_target_genes_version_2")
output_dfs_list = gene_content_analysis("drug_targets_v1_and_drug_targets_v2_set_difference_for_drug_targets_v2", gene_frequency_folder, "drug_target_genes_version_2_only")
output_dfs_list = gene_content_analysis("drug_targets_v1_and_drug_targets_v2_intersection", gene_frequency_folder, "drug_target_genes_version_1_and_2_intersecting")
output_dfs_list = gene_content_analysis("drug_targets_v1_and_drug_targets_v2_union", gene_frequency_folder, "drug_target_genes_version_1_and_2_intersecting")
output_dfs_list = gene_content_analysis("drug_targets_v1_and_drug_targets_v2_set_symmetric_difference", gene_frequency_folder, "drug_target_genes_version_1_and_2_symmetric_difference")

## Version Gene Count Comparison

In [12]:
# Obtaining overall frequency/count of a gene from all records

version_gene_count_comparison_folder = drug_target_gene_content_pathfx_version_comparison_info_folder + "version_gene_count_comparison/"
check_directory_exists(version_gene_count_comparison_folder)

df = test_all_drugs_super_association_df.copy(deep=True)
df["drug_targets_v1"] = df["drug_targets_v1"].apply(literal_eval)
df["drug_targets_v2"] = df["drug_targets_v2"].apply(literal_eval)
drug_target_gene_lst_v1 = df["drug_targets_v1"]
drug_target_gene_lst_v2 = df["drug_targets_v2"]
flattened_gene_lists_v1 = [item for sublist in drug_target_gene_lst_v1 for item in sublist]
flattened_gene_lists_v2 = [item for sublist in drug_target_gene_lst_v2 for item in sublist]
all_gene_list_dict_v1 = dict(Counter(flattened_gene_lists_v1))
all_gene_list_dict_v2 = dict(Counter(flattened_gene_lists_v2))
all_gene_list_df_v1 = pd.DataFrame.from_dict(all_gene_list_dict_v1, orient='index')
all_gene_list_df_v2 = pd.DataFrame.from_dict(all_gene_list_dict_v2, orient='index')
all_gene_list_df_v1.columns = ["All Records Version 1 Count"]
all_gene_list_df_v2.columns = ["All Records Version 2 Count"]
all_drug_target_genes_df = all_gene_list_df_v1.merge(all_gene_list_df_v2, how="outer", left_on=all_gene_list_df_v1.index, right_on=all_gene_list_df_v2.index)
all_drug_target_genes_df = all_drug_target_genes_df.rename(columns={"key_0": "Gene"})
all_drug_target_genes_df = all_drug_target_genes_df.fillna(0)
all_drug_target_genes_df = all_drug_target_genes_df.sort_values(by="Gene")

# Obtaining overall frequency/count of a gene for EACH adverse Event
df = test_all_drugs_super_association_df.copy(deep=True)
all_adverse_events = list(df["phenotype_v2"].unique())
all_adverse_events.sort()
genes_per_adverse_event_df = pd.DataFrame()
for adverse_event in all_adverse_events:
    df = test_all_drugs_super_association_df.copy(deep=True)
    df = df[df["phenotype_v2"] == adverse_event]
    df["drug_targets_v1"] = df["drug_targets_v1"].apply(literal_eval)
    df["drug_targets_v2"] = df["drug_targets_v2"].apply(literal_eval)
    drug_target_gene_lst_v1 = df["drug_targets_v1"]
    drug_target_gene_lst_v2 = df["drug_targets_v2"]
    flattened_gene_lists_v1 = [item for sublist in drug_target_gene_lst_v1 for item in sublist]
    flattened_gene_lists_v2 = [item for sublist in drug_target_gene_lst_v2 for item in sublist]
    all_gene_list_dict_v1 = dict(Counter(flattened_gene_lists_v1))
    all_gene_list_dict_v2 = dict(Counter(flattened_gene_lists_v2))
    all_gene_list_df_v1 = pd.DataFrame.from_dict(all_gene_list_dict_v1, orient='index')
    all_gene_list_df_v2 = pd.DataFrame.from_dict(all_gene_list_dict_v2, orient='index')
    all_gene_list_df_v1.columns = ["Version 1 Adverse Event Count"]
    all_gene_list_df_v2.columns = ["Version 2 Adverse Event Count"]
    all_adverse_event_drug_target_genes_df = all_gene_list_df_v1.merge(all_gene_list_df_v2, how="outer", left_on=all_gene_list_df_v1.index, right_on=all_gene_list_df_v2.index)
    all_adverse_event_drug_target_genes_df = all_adverse_event_drug_target_genes_df.rename(columns={"key_0": "Gene"})
    all_adverse_event_drug_target_genes_df = all_adverse_event_drug_target_genes_df.fillna(0)
    all_adverse_event_drug_target_genes_df["Adverse Event"] = adverse_event
    genes_per_adverse_event_df = pd.concat([genes_per_adverse_event_df, all_adverse_event_drug_target_genes_df], axis=0) 
genes_per_adverse_event_df = genes_per_adverse_event_df.sort_values(by="Gene")

gene_version_count_info_df = genes_per_adverse_event_df.merge(all_drug_target_genes_df, how="inner", left_on="Gene", right_on="Gene")

gene_version_count_info_df["Adverse Event Count Difference"] = gene_version_count_info_df["Version 2 Adverse Event Count"] - gene_version_count_info_df["Version 1 Adverse Event Count"]
gene_version_count_info_df["All Records Count Difference"] = gene_version_count_info_df["All Records Version 2 Count"] - gene_version_count_info_df["All Records Version 1 Count"]

save_to_csv_file(version_gene_count_comparison_folder, gene_version_count_info_df, "drug_target_genes_version_count_info_table", input_index=False)

gene_version_count_info_df

,Gene,Version 1 Adverse Event Count,Version 2 Adverse Event Count,Adverse Event,All Records Version 1 Count,All Records Version 2 Count,Adverse Event Count Difference,All Records Count Difference
0,ABCA3,1.0,1.0,Thrombocytopenia,1.0,1.0,0.0,0.0
1,ABCA5,1.0,1.0,Hyperlipidemia,1.0,1.0,0.0,0.0
2,ABCB1,7.0,6.0,Pancreatitis,119.0,119.0,-1.0,0.0
3,ABCB1,24.0,24.0,Pneumonia,119.0,119.0,0.0,0.0
4,ABCB1,22.0,23.0,Myocardial Infarction,119.0,119.0,1.0,0.0
...,...,...,...,...,...,...,...,...
1755,YES1,1.0,1.0,Thrombocytopenia,2.0,2.0,0.0,0.0
1756,YES1,1.0,1.0,Pulmonary Edema,2.0,2.0,0.0,0.0
1757,rpsD,0.0,1.0,Pancreatitis,0.0,1.0,1.0,1.0
1758,rpsI,0.0,1.0,Pancreatitis,0.0,1.0,1.0,1.0


In [13]:
gene_version_count_info_df = gene_version_count_info_df.sort_values(by="Adverse Event Count Difference", ascending=False)
graph_title = "Difference in Frequncy for Adverse Event Count between PathFX Versions for all Drug Target Genes"
fig = px.bar(data_frame=gene_version_count_info_df, x="Adverse Event Count Difference", y="Gene", color="Adverse Event", color_discrete_map=color_dict, 
             title=graph_title).update_xaxes(categoryorder="total descending").update_yaxes(categoryorder="total ascending")
fig.update_layout(showlegend=True, width=2700, height=1900)
fig.update_xaxes(title_text="Difference in Frequncy for Adverse Event Count between PathFX Versions")
fig.update_yaxes(title_text="Gene")
fig.update_layout(legend_title_text='Adverse Event')
fig.update_traces(
    hovertemplate="<br>".join([
        "Gene: %{y}",
        "Adverse Event Count Difference: %{x}"
    ])
)
fig.write_image(version_gene_count_comparison_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(version_gene_count_comparison_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

In [14]:
all_drug_target_genes_lst = list(set(gene_version_count_info_df["Gene"]))
len(all_drug_target_genes_lst)

491

In [24]:
lst = []
network = read_pickle_file("important_database_files/pfx041520_interactome.pkl")
count=0
for gene in all_drug_target_genes_lst:
    if gene not in network.nodes():
        print(gene)
        count=count+1
        lst.append(gene)
print()
print(count)

MAP3K20
Htr2c
GARS1
GLYATL2
SLC15A1
SLC36A1
SCN9A
CYP11B1
tynA
RIC3
CYP3A7
UGT2B17
SCN11A
CES2
rpsD
SLC22A7
rpsI
GLYATL1
SLC2A9
HSD17B2

20


In [17]:
import networkx as nx

input_interaction_data = 'important_database_files/scored_interactome_data_merged.txt'
input_interaction_data_df = pd.read_csv(input_interaction_data, sep="\t", header=None)
input_interaction_data_df = input_interaction_data_df.drop([3], axis=1)
input_interaction_data_df.columns = ["source", "target", "weight"]

input_interaction_data_df["source"] = input_interaction_data_df["source"].astype(str)
input_interaction_data_df["target"] = input_interaction_data_df["target"].astype(str)

source_nodes = list(input_interaction_data_df["source"].unique())
target_nodes = list(input_interaction_data_df["target"].unique())

if np.nan in source_nodes:
    source_nodes.remove(np.nan)

if np.nan in target_nodes:
    target_nodes.remove(np.nan)

all_nodes = list(set(source_nodes) | set(target_nodes))
all_nodes.sort()

input_interaction_data_modified_df = input_interaction_data_df.copy(deep=True)
input_interaction_data_modified_df["weight"] = pd.to_numeric(-1*np.log2(input_interaction_data_modified_df['weight']))

pathfx_interactome_network = nx.from_pandas_edgelist(input_interaction_data_df, source='source', target='target', edge_attr=True)
pathfx_interactome_network_modified = nx.from_pandas_edgelist(input_interaction_data_modified_df, source='source', target='target', edge_attr=True)

input_interaction_data_df = input_interaction_data_df.sort_values(by="weight", ascending=False)
display(input_interaction_data_df)

,source,target,weight
314972,rs11155012,ECT2L,0.990000
305428,rs10485828,TUBB1,0.990000
89821,HLA-B,HLA-B*46:01:01,0.990000
89819,HLA-B,HLA-B*13:02:01,0.990000
89760,rs3918290,DPYD,0.990000
...,...,...,...
47545,rs28401781,sertraline,0.233675
47544,rs28401781,fluoxetine,0.233675
47542,rs28401781,paroxetine,0.233675
47541,rs28401781,citalopram,0.233675


In [23]:
network = read_pickle_file("important_database_files/pfx041520_interactome.pkl")
count=0
for gene in all_drug_target_genes_lst:
    if gene not in list(input_interaction_data_df["source"]) and gene not in list(input_interaction_data_df["target"]):
        print(gene)
        count=count+1
print()
print(count)

MAP3K20
Htr2c
GARS1
GLYATL2
SLC15A1
SLC36A1
SCN9A
CYP11B1
tynA
RIC3
CYP3A7
UGT2B17
SCN11A
CES2
rpsD
SLC22A7
rpsI
GLYATL1
SLC2A9
HSD17B2

20


In [25]:
lst.sort()
textfile = open("a_file.txt", "w")
for element in lst:
    textfile.write(element + "\n")
textfile.close()